## Prepare Data

In [25]:
# โหลดข้อมูล, ไลบรารี
import pandas as pd
import geojson

prov_filter = ['ชัยภูมิ','นครราชสีมา','นครสวรรค์','พิจิตร', 'เพชรบูรณ์', 'ลพบุรี']

# ข้อมูลดิบ
df_sheet1 = pd.read_excel("sheet1.xlsx")
df_sheet1 = df_sheet1.rename(columns={"ลำดับ": "ลำดับแปลง"})

# ข้อมูลน้ำบาดาลรัฐ
df_ckan_wells2 = pd.read_excel("Ckan_wells2.xlsx")
df_ckan_wells2 = df_ckan_wells2.sort_values(by='LATITUDE', ascending=False)
df_ckan_wells2.drop(df_ckan_wells2.tail(1).index,inplace=True) # drop last row เพราะเป็นค่าว่าง
df_ckan_wells2['Index'] = range(1, len(df_ckan_wells2) + 1)
df_ckan_wells2 = df_ckan_wells2[df_ckan_wells2['ชื่อจังหวัด'].isin(prov_filter)]

## หาทิศจากสูตร Bearing Direction

<p style="text-align:center;"> 
<img src="https://elearning.reb.rw/pluginfile.php/26152/course/section/7273/43.PNG" width=500 display= block margin-left=auto margin-right=auto><br></br>

<img src="https://raw.githubusercontent.com/Thanarat-DS/Thanarat-DS/main/Asset/Bearing.png" width=800>
</p>

In [26]:
import numpy as np

def bearing_direction(lat1, lon1, lat2, lon2):
    # Convert degrees to radians
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)

    dlon = lon2_rad - lon1_rad

    # Calculate bearing using atan2
    y = np.sin(dlon) * np.cos(lat2_rad)
    x = np.cos(lat1_rad) * np.sin(lat2_rad) - np.sin(lat1_rad) * np.cos(lat2_rad) * np.cos(dlon)
    bearing_rad = np.arctan2(y, x)

    # Convert bearing from radians to degrees
    bearing_deg = np.degrees(bearing_rad)

    # Normalize to a compass bearing (0 - 360 degrees)
    bearing_normalized = (bearing_deg + 360) % 360

    directions = ["เหนือ", "ตะวันออกเฉียงเหนือ", "ตะวันออก", "ตะวันออกเฉียงใต้", "ใต้", "ตะวันตกเฉียงใต้", "ตะวันตก", "ตะวันตกเฉียงเหนือ"]
    index = int((bearing_normalized + 22.5) / 45) % 8
    compass_direction = directions[index]

    return bearing_normalized, compass_direction


<p style="text-align:center;"> 
<img src="https://raw.githubusercontent.com/Thanarat-DS/Thanarat-DS/main/Asset/direction.jpg">
</p>

### หาบ่อบาดาลในระยะ 2 กิโลเมตร
<p style="text-align:center">
<img src="https://michaelminn.net/tutorials/gis-distance/2022-haversine-formula.png"/>
</p>

In [27]:
def haversine_vectorized(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in kilometers

    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)
    
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    
    a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    distance = R * c

    return distance

# Function to find wells within 2 km radius
def find_closest_wells_vectorized(lat1, lon1, df_wells):
    distances = haversine_vectorized(lat1, lon1, df_wells['LATITUDE'], df_wells['LONGITUDE'])
    mask = distances <= 2.0 #ผลลัพธ์เป็น List Boolean เช่น [True, False, True, True, False]
    closest_wells = df_wells[mask]['Index'].tolist()
    closest_distances = distances[mask]
    bearings = []
    directions = []
    for lat2, lon2 in zip(df_wells[mask]['LATITUDE'], df_wells[mask]['LONGITUDE']):
        bearing, direction = bearing_direction(lat1, lon1, lat2, lon2)
        bearings.append(bearing)
        directions.append(direction)

    return closest_wells, closest_distances, bearings, directions

# Create a new DataFrame to store the results
result_data = []

# Iterate over rows in df_sheet1
for index, row in df_sheet1.iterrows():
    closest_wells, closest_distances, closest_bearings, closest_directions = find_closest_wells_vectorized(row['Latitude'], row['Longitude'], df_ckan_wells2)

    if len(closest_wells) > 0:
        # Append rows for each closest well found
        for well, dist, bearing, direction in zip(closest_wells, closest_distances, closest_bearings, closest_directions):
            result_data.append([row['ลำดับแปลง'], row['Latitude'], row['Longitude'], row['ตัน/ไร่ประเมิน67/68'], row['ตัน/ไร่เกิดจริงปี66/67'], well, dist, bearing, direction])

    # Code เก่า
    # if len(closest_wells) == 0:
    #     # If no wells are found within 2 km, append a new row with placeholder values
    #     result_data.append([row['ลำดับแปลง'], row['Latitude'], row['Longitude'], row['ตัน/ไร่ประเมิน67/68'], row['ตัน/ไร่เกิดจริงปี66/67'], None, None, None, None])
    # else:
    #     # Append rows for each closest well found
    #     for well, dist, bearing, direction in zip(closest_wells, closest_distances, closest_bearings, closest_directions):
    #         result_data.append([row['ลำดับแปลง'], row['Latitude'], row['Longitude'], row['ตัน/ไร่ประเมิน67/68'], row['ตัน/ไร่เกิดจริงปี66/67'], well, dist, bearing, direction])

# Convert result_data to DataFrame
result_df = pd.DataFrame(result_data, columns=['ลำดับแปลง', 'Latitude', 'Longitude', 'ตัน/ไร่ประเมิน67/68', 'ตัน/ไร่เกิดจริงปี66/67', 'Closest_Well', 'Distance', 'Bearing', 'Direction'])
result_df.describe()


,ลำดับแปลง,Latitude,Longitude,ตัน/ไร่ประเมิน67/68,Closest_Well,Distance,Bearing
count,22850.000000,22850.000000,22850.000000,22850.000000,22850.000000,22850.000000,22850.000000
mean,4700.679606,15.542708,101.056304,9.100460,36979.736718,1.343079,179.660685
std,2708.567730,0.181911,0.177585,1.750082,2601.649909,0.454751,103.169848
min,1.000000,14.983425,100.661132,3.000000,28318.000000,0.025521,0.011552
25%,2366.000000,15.400615,100.940685,8.000000,35466.000000,1.023678,93.208074
50%,4748.500000,15.559007,101.001430,9.000000,36931.000000,1.415796,177.360851
75%,7031.750000,15.662022,101.167133,10.000000,38717.000000,1.727209,267.228417
max,9353.000000,16.073873,101.945793,15.000000,46569.000000,1.999993,359.927687


### Merge และ Clean ชื่อคอลัมน์

In [28]:
result_df_merge = result_df.merge(df_ckan_wells2,how='left', left_on='Closest_Well', right_on='Index')
result_df_merge = result_df_merge.rename(columns={
    #"ชื่อตำบล": "Sub_District",
    #"ชื่ออำเภอ": "District" ,
    #"ชื่อจังหวัด": "Province",
    #"ประเภทบ่อน้ำบาดาล": "Well_Type",
    #"สถานะบ่อ": "Well_Status",
    #"สภาพน้ำบาดาล": "Groundwater Conditions",
    #"ความลึกเจาะ": "DeepthDrill",
    #"ความลึกพัฒนา": "DeepDevelop",
    #"ปริมาณน้ำ": "Yield",
    "LATITUDE": "Latitude_น้ำบาดาล",
    "LONGITUDE": "Longitude_น้ำบาดาล"
})

result_df_merge = result_df_merge.drop(columns=['Index'])
result_df_merge = result_df_merge.sort_values(by=['ลำดับแปลง', 'Distance'], ascending=True)
result_df_merge['แปลงไร่/บ่อน้ำบาดาล'] = 'บ่อน้ำบาดาล'
display(result_df_merge)

,ลำดับแปลง,Latitude,Longitude,ตัน/ไร่ประเมิน67/68,ตัน/ไร่เกิดจริงปี66/67,Closest_Well,Distance,Bearing,Direction,หมายเลขบ่อน้ำบาดาล,...,ชื่อจังหวัด,ประเภทบ่อน้ำบาดาล,สถานะบ่อ,สภาพน้ำบาดาล,ความลึกเจาะ,ความลึกพัฒนา,ปริมาณน้ำ,Latitude_น้ำบาดาล,Longitude_น้ำบาดาล,แปลงไร่/บ่อน้ำบาดาล
1,1,15.626392,100.967404,12.0,ไม่ได้ต่อสัญญาปี66/67,35912,0.399201,347.223102,เหนือ,\t'N1063',...,เพชรบูรณ์,อุปโภค-บริโภค,ใช้การได้,น้ำจืด,36.0,36.0,12.95,15.629893,100.966579,บ่อน้ำบาดาล
2,1,15.626392,100.967404,12.0,ไม่ได้ต่อสัญญาปี66/67,35927,0.965742,71.614633,ตะวันออก,\t'N461',...,เพชรบูรณ์,อุปโภค-บริโภค,ใช้การได้,น้ำจืด,18.0,18.0,1.59,15.629131,100.975962,บ่อน้ำบาดาล
3,1,15.626392,100.967404,12.0,ไม่ได้ต่อสัญญาปี66/67,35941,1.498224,81.324697,ตะวันออก,\t'6303M017',...,เพชรบูรณ์,อุปโภค-บริโภค,ใช้การได้,น้ำจืด,104.0,104.0,4.60,15.628424,100.981235,บ่อน้ำบาดาล
0,1,15.626392,100.967404,12.0,ไม่ได้ต่อสัญญาปี66/67,35791,1.535912,326.891342,ตะวันตกเฉียงเหนือ,\t'6103E042',...,เพชรบูรณ์,อุปโภค-บริโภค,ใช้การได้,น้ำจืด,75.0,75.0,6.00,15.637962,100.959569,บ่อน้ำบาดาล
5,1,15.626392,100.967404,12.0,ไม่ได้ต่อสัญญาปี66/67,36156,1.620289,166.183717,ใต้,\t'5803H029',...,เพชรบูรณ์,เกษตร,ใช้การได้,น้ำจืด,80.0,80.0,8.00,15.612242,100.971017,บ่อน้ำบาดาล
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22845,9350,15.444615,100.890814,11.0,ไม่ได้ต่อสัญญาปี66/67,38203,1.155868,260.589732,ตะวันตก,\t'P975',...,ลพบุรี,อุปโภค-บริโภค,ใช้การได้,น้ำจืด,30.0,30.0,2.30,15.442915,100.880174,บ่อน้ำบาดาล
22846,9351,15.812013,100.840518,12.0,7.437579,32928,1.843356,328.726699,ตะวันตกเฉียงเหนือ,\t'TZ215',...,เพชรบูรณ์,อุปโภค-บริโภค,ใช้การได้,น้ำจืด,42.0,42.0,6.00,15.826182,100.831573,บ่อน้ำบาดาล
22847,9352,15.750852,101.107766,9.0,10.455916,34307,1.327583,196.735371,ใต้,\t'TZ9',...,เพชรบูรณ์,อุปโภค-บริโภค,ใช้การได้,น้ำจืด,96.0,81.0,6.80,15.739418,101.104194,บ่อน้ำบาดาล
22848,9352,15.750852,101.107766,9.0,10.455916,34353,1.956910,209.234318,ตะวันตกเฉียงใต้,\t'N1229',...,เพชรบูรณ์,อุปโภค-บริโภค,ใช้การได้,น้ำจืด,87.0,87.0,6.81,15.735494,101.098836,บ่อน้ำบาดาล


### จัดอันดับบ่อที่ใกล้ที่สุด (Ranking)

In [29]:
ranking_list = []
last_index = 0
running_number = 0 

for index, row in result_df_merge.iterrows():
    if row['ลำดับแปลง'] > last_index:
        running_number = 1 # Reset Ranking
        ranking_list.append(running_number)
    else:
        ranking_list.append(running_number)

    running_number += 1 # Auto Running จนกว่าจะขึ้นเลขลำดับใหม่
    last_index = row['ลำดับแปลง']

result_df_merge['อันดับบ่อที่ใกล้ที่สุด'] = ranking_list
result_df_merge

,ลำดับแปลง,Latitude,Longitude,ตัน/ไร่ประเมิน67/68,ตัน/ไร่เกิดจริงปี66/67,Closest_Well,Distance,Bearing,Direction,หมายเลขบ่อน้ำบาดาล,...,ประเภทบ่อน้ำบาดาล,สถานะบ่อ,สภาพน้ำบาดาล,ความลึกเจาะ,ความลึกพัฒนา,ปริมาณน้ำ,Latitude_น้ำบาดาล,Longitude_น้ำบาดาล,แปลงไร่/บ่อน้ำบาดาล,อันดับบ่อที่ใกล้ที่สุด
1,1,15.626392,100.967404,12.0,ไม่ได้ต่อสัญญาปี66/67,35912,0.399201,347.223102,เหนือ,\t'N1063',...,อุปโภค-บริโภค,ใช้การได้,น้ำจืด,36.0,36.0,12.95,15.629893,100.966579,บ่อน้ำบาดาล,1
2,1,15.626392,100.967404,12.0,ไม่ได้ต่อสัญญาปี66/67,35927,0.965742,71.614633,ตะวันออก,\t'N461',...,อุปโภค-บริโภค,ใช้การได้,น้ำจืด,18.0,18.0,1.59,15.629131,100.975962,บ่อน้ำบาดาล,2
3,1,15.626392,100.967404,12.0,ไม่ได้ต่อสัญญาปี66/67,35941,1.498224,81.324697,ตะวันออก,\t'6303M017',...,อุปโภค-บริโภค,ใช้การได้,น้ำจืด,104.0,104.0,4.60,15.628424,100.981235,บ่อน้ำบาดาล,3
0,1,15.626392,100.967404,12.0,ไม่ได้ต่อสัญญาปี66/67,35791,1.535912,326.891342,ตะวันตกเฉียงเหนือ,\t'6103E042',...,อุปโภค-บริโภค,ใช้การได้,น้ำจืด,75.0,75.0,6.00,15.637962,100.959569,บ่อน้ำบาดาล,4
5,1,15.626392,100.967404,12.0,ไม่ได้ต่อสัญญาปี66/67,36156,1.620289,166.183717,ใต้,\t'5803H029',...,เกษตร,ใช้การได้,น้ำจืด,80.0,80.0,8.00,15.612242,100.971017,บ่อน้ำบาดาล,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22845,9350,15.444615,100.890814,11.0,ไม่ได้ต่อสัญญาปี66/67,38203,1.155868,260.589732,ตะวันตก,\t'P975',...,อุปโภค-บริโภค,ใช้การได้,น้ำจืด,30.0,30.0,2.30,15.442915,100.880174,บ่อน้ำบาดาล,1
22846,9351,15.812013,100.840518,12.0,7.437579,32928,1.843356,328.726699,ตะวันตกเฉียงเหนือ,\t'TZ215',...,อุปโภค-บริโภค,ใช้การได้,น้ำจืด,42.0,42.0,6.00,15.826182,100.831573,บ่อน้ำบาดาล,1
22847,9352,15.750852,101.107766,9.0,10.455916,34307,1.327583,196.735371,ใต้,\t'TZ9',...,อุปโภค-บริโภค,ใช้การได้,น้ำจืด,96.0,81.0,6.80,15.739418,101.104194,บ่อน้ำบาดาล,1
22848,9352,15.750852,101.107766,9.0,10.455916,34353,1.956910,209.234318,ตะวันตกเฉียงใต้,\t'N1229',...,อุปโภค-บริโภค,ใช้การได้,น้ำจืด,87.0,87.0,6.81,15.735494,101.098836,บ่อน้ำบาดาล,2


### ทำให้ Plot แผนที่ได้
(รวมคอลัมน์ Latitude, Longitude ทั้ง 2 ตารางด้วยการ Concat กัน)

In [30]:
clone_df_sheet1 = df_sheet1
clone_df_sheet1['แปลงไร่/บ่อน้ำบาดาล'] = 'แปลงไร่'
clone_df_sheet1 = clone_df_sheet1.rename(columns={
    "Longitude": "Longitude_น้ำบาดาล",
    "Latitude": "Latitude_น้ำบาดาล"
})

concat_df = pd.concat([clone_df_sheet1, result_df_merge], ignore_index=True).sort_values(by='ลำดับแปลง', ascending=True)
concat_df = concat_df.drop(columns=['Latitude','Longitude'])
concat_df = concat_df.rename(columns={
    "Longitude_น้ำบาดาล": "Longitude",
    "Latitude_น้ำบาดาล": "Latitude"
})
concat_df = concat_df.replace(np.nan, None)
concat_df

,ลำดับแปลง,Latitude,Longitude,ตัน/ไร่ประเมิน67/68,ตัน/ไร่เกิดจริงปี66/67,แปลงไร่/บ่อน้ำบาดาล,Closest_Well,Distance,Bearing,Direction,...,ชื่อตำบล,ชื่ออำเภอ,ชื่อจังหวัด,ประเภทบ่อน้ำบาดาล,สถานะบ่อ,สภาพน้ำบาดาล,ความลึกเจาะ,ความลึกพัฒนา,ปริมาณน้ำ,อันดับบ่อที่ใกล้ที่สุด
0,1,15.626392,100.967404,12.0,ไม่ได้ต่อสัญญาปี66/67,แปลงไร่,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9287,1,15.612242,100.971017,12.0,ไม่ได้ต่อสัญญาปี66/67,บ่อน้ำบาดาล,36156.0,1.620289,166.183717,ใต้,...,วังใหญ่,หล่มเก่า,เพชรบูรณ์,เกษตร,ใช้การได้,น้ำจืด,80.0,80.0,8.0,5.0
9286,1,15.637962,100.959569,12.0,ไม่ได้ต่อสัญญาปี66/67,บ่อน้ำบาดาล,35791.0,1.535912,326.891342,ตะวันตกเฉียงเหนือ,...,วังใหญ่,วิเชียรบุรี,เพชรบูรณ์,อุปโภค-บริโภค,ใช้การได้,น้ำจืด,75.0,75.0,6.0,4.0
9285,1,15.628424,100.981235,12.0,ไม่ได้ต่อสัญญาปี66/67,บ่อน้ำบาดาล,35941.0,1.498224,81.324697,ตะวันออก,...,วังใหญ่,วิเชียรบุรี,เพชรบูรณ์,อุปโภค-บริโภค,ใช้การได้,น้ำจืด,104.0,104.0,4.6,3.0
9288,1,15.620408,100.950022,12.0,ไม่ได้ต่อสัญญาปี66/67,บ่อน้ำบาดาล,36060.0,1.976714,250.332975,ตะวันตก,...,วังใหญ่,วิเชียรบุรี,เพชรบูรณ์,อุปโภค-บริโภค,ใช้การได้,น้ำจืด,30.0,30.0,1.59,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32131,9352,15.735494,101.098836,9.0,10.455916,บ่อน้ำบาดาล,34353.0,1.95691,209.234318,ตะวันตกเฉียงใต้,...,บึงกระจับ,วิเชียรบุรี,เพชรบูรณ์,อุปโภค-บริโภค,ใช้การได้,น้ำจืด,87.0,87.0,6.81,2.0
9281,9352,15.750852,101.107766,9.0,10.455916,แปลงไร่,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
32130,9352,15.739418,101.104194,9.0,10.455916,บ่อน้ำบาดาล,34307.0,1.327583,196.735371,ใต้,...,บึงกระจับ,วิเชียรบุรี,เพชรบูรณ์,อุปโภค-บริโภค,ใช้การได้,น้ำจืด,96.0,81.0,6.8,1.0
32132,9353,15.665073,101.157249,7.0,ไม่ได้ต่อสัญญาปี66/67,บ่อน้ำบาดาล,35401.0,1.511035,193.877616,ใต้,...,ท่าโรง,วิเชียรบุรี,เพชรบูรณ์,อุปโภค-บริโภค,ใช้การได้,น้ำจืด,51.0,51.0,2.95,1.0


## DataFrame to GeoJson

#### ข้อมูลแปลงไร่

In [26]:
# สร้าง GeoJSON features จาก DataFrame
features = []
for _, row in df_sheet1.iterrows():
    feature = geojson.Feature(
        geometry=geojson.Point((row['Longitude'], row['Latitude'])),
        properties={
            'ลำดับแปลง': row['ลำดับแปลง'],
            'ตัน/ไร่ประเมิน67/68': row['ตัน/ไร่ประเมิน67/68'],
            'ตัน/ไร่เกิดจริงปี66/67': row['ตัน/ไร่เกิดจริงปี66/67']
        }
    )
    features.append(feature)

# สร้าง GeoJSON FeatureCollection
feature_collection = geojson.FeatureCollection(features)

# แปลงเป็น string หรือบันทึกเป็นไฟล์ GeoJSON
geojson_str = geojson.dumps(feature_collection, ensure_ascii=False, indent=2)
with open('output.json', 'w', encoding='utf-8') as f:
    f.write(geojson_str)
print(geojson_str[:1000])  # แสดงเฉพาะ 1000 ตัวอักษรแรก

{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          100.967404,
          15.626392
        ]
      },
      "properties": {
        "ลำดับแปลง": 1,
        "ตัน/ไร่ประเมิน67/68": 12.0,
        "ตัน/ไร่เกิดจริงปี66/67": "ไม่ได้ต่อสัญญาปี66/67"
      }
    },
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          100.911557,
          15.422767
        ]
      },
      "properties": {
        "ลำดับแปลง": 2,
        "ตัน/ไร่ประเมิน67/68": 10.0,
        "ตัน/ไร่เกิดจริงปี66/67": "ไม่ได้ต่อสัญญาปี66/67"
      }
    },
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          100.914545,
          15.692258
        ]
      },
      "properties": {
        "ลำดับแปลง": 3,
        "ตัน/ไร่ประเมิน67/68": 12.0,
        "ตัน/ไร่เกิดจริงปี66/67": "ไม่ได้ต่อสัญญาปี66/67"
      }
    

#### ข้อมูลบ่อบาดาลจากรัฐบาล

In [8]:
df_ckan_wells2.iloc[0,0]

"\t'PW15334'"

In [31]:
outliers_list = ["\t'5901B027'","\t'5901B027'"]
outliers_list[1]

"\t'5901B027'"

In [25]:
prov_list = ['ชัยภูมิ','นครราชสีมา','นครสวรรค์','พิจิตร', 'เพชรบูรณ์', 'ลพบุรี']
df_ckan_wells_filter = df_ckan_wells2[df_ckan_wells2['ชื่อจังหวัด'].isin(prov_list)]

# สร้าง GeoJSON features จาก DataFrame
features = []
for _, row in df_ckan_wells_filter.iterrows():
    feature = geojson.Feature(
        geometry=geojson.Point((row['LONGITUDE'], row['LATITUDE'])),
        properties={
            'หมายเลขบ่อน้ำบาดาล': row['หมายเลขบ่อน้ำบาดาล'],
            'ชื่อตำบล': row['ชื่อตำบล'],
            'ชื่ออำเภอ': row['ชื่ออำเภอ'],
            'ชื่อจังหวัด': row['ชื่อจังหวัด'],
            'ประเภทบ่อน้ำบาดาล': row['ประเภทบ่อน้ำบาดาล'],
            'ความลึกเจาะ': row['ความลึกเจาะ'],
            'ความลึกพัฒนา': row['ความลึกพัฒนา'],
            'ปริมาณน้ำ': row['ปริมาณน้ำ']
        }
    )
    features.append(feature)

# สร้าง GeoJSON FeatureCollection
feature_collection = geojson.FeatureCollection(features)

# แปลงเป็น string หรือบันทึกเป็นไฟล์ GeoJSON
geojson_str = geojson.dumps(feature_collection, ensure_ascii=False, indent=2)
with open('output_groundwater.json', 'w', encoding='utf-8') as f:
    f.write(geojson_str)
print(geojson_str[:1000])  # แสดงเฉพาะ 1000 ตัวอักษรแรก

{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          101.391448,
          17.101176
        ]
      },
      "properties": {
        "หมายเลขบ่อน้ำบาดาล": "\t'PW15334'",
        "ชื่อตำบล": "ศิลา",
        "ชื่ออำเภอ": "หล่มเก่า",
        "ชื่อจังหวัด": "เพชรบูรณ์",
        "ประเภทบ่อน้ำบาดาล": "อุปโภค-บริโภค",
        "ความลึกเจาะ": 60.0,
        "ความลึกพัฒนา": 60.0,
        "ปริมาณน้ำ": 1.5
      }
    },
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          101.391448,
          17.101176
        ]
      },
      "properties": {
        "หมายเลขบ่อน้ำบาดาล": "\t'N599'",
        "ชื่อตำบล": "ศิลา",
        "ชื่ออำเภอ": "หล่มเก่า",
        "ชื่อจังหวัด": "เพชรบูรณ์",
        "ประเภทบ่อน้ำบาดาล": "อุปโภค-บริโภค",
        "ความลึกเจาะ": 36.0,
        "ความลึกพัฒนา": 36.0,
        "ปริมาณน้ำ": 3.18
      }
    },
    {
     

#### ข้อมูลหลัง Merged

In [23]:
concat_df

,ลำดับแปลง,Latitude,Longitude,ตัน/ไร่ประเมิน67/68,ตัน/ไร่เกิดจริงปี66/67,แปลงไร่/บ่อน้ำบาดาล,Closest_Well,Distance,Bearing,Direction,...,ชื่อตำบล,ชื่ออำเภอ,ชื่อจังหวัด,ประเภทบ่อน้ำบาดาล,สถานะบ่อ,สภาพน้ำบาดาล,ความลึกเจาะ,ความลึกพัฒนา,ปริมาณน้ำ,อันดับบ่อที่ใกล้ที่สุด
0,1,15.626392,100.967404,12.0,ไม่ได้ต่อสัญญาปี66/67,แปลงไร่,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9283,1,15.629893,100.966579,12.0,ไม่ได้ต่อสัญญาปี66/67,บ่อน้ำบาดาล,35912.0,0.399201,347.223102,เหนือ,...,วังใหญ่,วิเชียรบุรี,เพชรบูรณ์,อุปโภค-บริโภค,ใช้การได้,น้ำจืด,36.0,36.0,12.95,1.0
9285,1,15.628424,100.981235,12.0,ไม่ได้ต่อสัญญาปี66/67,บ่อน้ำบาดาล,35941.0,1.498224,81.324697,ตะวันออก,...,วังใหญ่,วิเชียรบุรี,เพชรบูรณ์,อุปโภค-บริโภค,ใช้การได้,น้ำจืด,104.0,104.0,4.6,3.0
9284,1,15.629131,100.975962,12.0,ไม่ได้ต่อสัญญาปี66/67,บ่อน้ำบาดาล,35927.0,0.965742,71.614633,ตะวันออก,...,วังใหญ่,วิเชียรบุรี,เพชรบูรณ์,อุปโภค-บริโภค,ใช้การได้,น้ำจืด,18.0,18.0,1.59,2.0
9288,1,15.620408,100.950022,12.0,ไม่ได้ต่อสัญญาปี66/67,บ่อน้ำบาดาล,36060.0,1.976714,250.332975,ตะวันตก,...,วังใหญ่,วิเชียรบุรี,เพชรบูรณ์,อุปโภค-บริโภค,ใช้การได้,น้ำจืด,30.0,30.0,1.59,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34660,9352,15.735494,101.098836,9.0,10.455916,บ่อน้ำบาดาล,34353.0,1.95691,209.234318,ตะวันตกเฉียงใต้,...,บึงกระจับ,วิเชียรบุรี,เพชรบูรณ์,อุปโภค-บริโภค,ใช้การได้,น้ำจืด,87.0,87.0,6.81,2.0
34659,9352,15.739418,101.104194,9.0,10.455916,บ่อน้ำบาดาล,34307.0,1.327583,196.735371,ใต้,...,บึงกระจับ,วิเชียรบุรี,เพชรบูรณ์,อุปโภค-บริโภค,ใช้การได้,น้ำจืด,96.0,81.0,6.8,1.0
9281,9352,15.750852,101.107766,9.0,10.455916,แปลงไร่,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9282,9353,15.678266,101.160634,7.0,ไม่ได้ต่อสัญญาปี66/67,แปลงไร่,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [24]:
concat_df.loc[:,'Latitude'].sort_values()

27124    14.968321
27123    14.968321
6572     14.983425
27122    14.992896
11718    15.026314
           ...    
34621         None
34633         None
34636         None
34638         None
34639         None
Name: Latitude, Length: 34662, dtype: object

In [31]:
# สร้าง GeoJSON features จาก DataFrame
features = []
for _, row in concat_df.iterrows():
    feature = geojson.Feature(
        geometry=geojson.Point((row['Longitude'], row['Latitude'])),
        properties={
            'ลำดับแปลง': row['ลำดับแปลง'],
            'ตัน/ไร่ประเมิน67/68': row['ตัน/ไร่ประเมิน67/68'],
            'ตัน/ไร่เกิดจริงปี66/67': row['ประเภทบ่อน้ำบาดาล'],
            'แปลงไร่/บ่อน้ำบาดาล': row['แปลงไร่/บ่อน้ำบาดาล'],
            'Closest_Well': row['Closest_Well'],
            'Distance': row['Distance'],
            'Bearing': row['Bearing'],
            'Direction': row['Direction'],
            'หมายเลขบ่อน้ำบาดาล': row['หมายเลขบ่อน้ำบาดาล'],
            'ชื่อตำบล': row['ชื่อตำบล'],
            'ชื่ออำเภอ': row['ชื่ออำเภอ'],
            'ชื่อจังหวัด': row['ชื่อจังหวัด'],
            'ประเภทบ่อน้ำบาดาล': row['ประเภทบ่อน้ำบาดาล'],
            'สถานะบ่อ': row['สถานะบ่อ'],
            'สภาพน้ำบาดาล': row['สภาพน้ำบาดาล'],
            'ความลึกเจาะ': row['ความลึกเจาะ'],
            'ความลึกพัฒนา': row['ความลึกพัฒนา'],
            'ปริมาณน้ำ': row['ปริมาณน้ำ'],
            'อันดับบ่อที่ใกล้ที่สุด': row['อันดับบ่อที่ใกล้ที่สุด']

        }
    )
    features.append(feature)

# สร้าง GeoJSON FeatureCollection
feature_collection = geojson.FeatureCollection(features)

# แปลงเป็น string หรือบันทึกเป็นไฟล์ GeoJSON
geojson_str = geojson.dumps(feature_collection, ensure_ascii=False, indent=2)
with open('merged_data.json', 'w', encoding='utf-8') as f:
    f.write(geojson_str)
print(geojson_str[:1000])  # แสดงเฉพาะ 1000 ตัวอักษรแรก

{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          100.967404,
          15.626392
        ]
      },
      "properties": {
        "ลำดับแปลง": 1,
        "ตัน/ไร่ประเมิน67/68": 12.0,
        "ตัน/ไร่เกิดจริงปี66/67": null,
        "แปลงไร่/บ่อน้ำบาดาล": "แปลงไร่",
        "Closest_Well": null,
        "Distance": null,
        "Bearing": null,
        "Direction": null,
        "หมายเลขบ่อน้ำบาดาล": null,
        "ชื่อตำบล": null,
        "ชื่ออำเภอ": null,
        "ชื่อจังหวัด": null,
        "ประเภทบ่อน้ำบาดาล": null,
        "สถานะบ่อ": null,
        "สภาพน้ำบาดาล": null,
        "ความลึกเจาะ": null,
        "ความลึกพัฒนา": null,
        "ปริมาณน้ำ": null,
        "อันดับบ่อที่ใกล้ที่สุด": null
      }
    },
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          100.971017,
          15.612242
        ]
      },
     